<a href="https://colab.research.google.com/github/steelpipe75/kagglebook-for-colab/blob/master/ch04-model-interface/code/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
import importlib
import sys
import subprocess

# Google Colab 上で実行しているかどうかを判断するフラグ
ON_COLAB = "google.colab" in sys.modules
print(f"ON_COLAB: {ON_COLAB}")

if ON_COLAB:
    # USE_KAGGLE_GIT = True # Kaggle-cli,Gitを使う
    USE_KAGGLE_GIT = False # Kaggle-cli,Gitを使わない

    import sys
    import os
    import zipfile

    os.makedirs('model', exist_ok=True)

    print(f"USE_KAGGLE_GIT: {USE_KAGGLE_GIT}")
    if USE_KAGGLE_GIT:
        import json
        from google.colab import userdata
        # シークレットからKAGGLE_USERNAMEとKAGGLE_KEYを取得
        KAGGLE_USERNAME = userdata.get('KAGGLE_USERNAME')
        KAGGLE_KEY = userdata.get('KAGGLE_KEY')

        # kaggle.jsonファイルを作成
        kaggle_credentials = {"username": KAGGLE_USERNAME, "key": KAGGLE_KEY}
        with open("kaggle.json", "w") as f:
            json.dump(kaggle_credentials, f)

        # kaggle.jsonファイルを~/.kaggleディレクトリに移動し、パーミッションを設定
        !mkdir -p ~/.kaggle
        !mv kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json

        # Kaggle APIをインストール
        %pip install kaggle
        !kaggle competitions download -c otto-group-product-classification-challenge

        zip_path = '/content/otto-group-product-classification-challenge.zip'

        !git clone https://github.com/steelpipe75/kagglebook-for-colab.git

        sys.path.append('/content/kagglebook-for-colab/ch04-model-interface/code')
    else:
        # Google Drive にマウントする
        drive = importlib.import_module("google.colab.drive")
        drive.mount("/content/drive/")

        colab_dir = "/content/drive/MyDrive/kagglebook/" # データ置き場
        sys.path.append(os.path.join(colab_dir, 'ch04-model-interface/code'))

        zip_path = os.path.join(colab_dir, 'ch04-model-interface/input/otto-group-product-classification-challenge.zip')

    extract_to = '/content/input/'
    os.makedirs(extract_to, exist_ok=True)

    # titanic.zipが存在するか確認
    if os.path.exists(zip_path):
        # titanic.zipを展開
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"{zip_path}を展開しました。")
    else:
        print(f"{zip_path}が見つかりません。")

ON_COLAB: True
USE_KAGGLE_GIT: False
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/kagglebook/ch04-model-interface/input/otto-group-product-classification-challenge.zipを展開しました。


In [3]:
from model_nn import ModelNN
from model_xgb import ModelXGB
from runner import Runner
from util import Submission

In [4]:
if __name__ == '__main__':

    params_xgb = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 9,
        'max_depth': 12,
        'eta': 0.1,
        'min_child_weight': 10,
        'subsample': 0.9,
        'colsample_bytree': 0.8,
        'silent': 1,
        'random_state': 71,
        'num_round': 10000,
        'early_stopping_rounds': 10,
    }

    params_xgb_all = dict(params_xgb)
    params_xgb_all['num_round'] = 350

    params_nn = {
        'layers': 3,
        # サンプルのため早く終わるように設定
        'nb_epoch': 5,  # 1000
        'patience': 10,
        'dropout': 0.5,
        'units': 512,
    }

    # 特徴量の指定
    features = [f'feat_{i}' for i in range(1, 94)]

    # xgboostによる学習・予測
    runner = Runner('xgb1', ModelXGB, features, params_xgb)
    runner.run_train_cv()
    runner.run_predict_cv()
    Submission.create_submission('xgb1')

    # ニューラルネットによる学習・予測
    runner = Runner('nn1', ModelNN, features, params_nn)
    runner.run_train_cv()
    runner.run_predict_cv()
    Submission.create_submission('nn1')

    '''
    # (参考）xgboostによる学習・予測 - 学習データ全体を使う場合
    runner = Runner('xgb1-train-all', ModelXGB, features, params_xgb_all)
    runner.run_train_all()
    runner.run_test_all()
    Submission.create_submission('xgb1-train-all')
    '''

[2025-04-07 11:15:26] - xgb1 - start training cv
INFO:general:[2025-04-07 11:15:26] - xgb1 - start training cv
[2025-04-07 11:15:26] - xgb1 fold 0 - start training
INFO:general:[2025-04-07 11:15:26] - xgb1 fold 0 - start training


FileNotFoundError: [Errno 2] No such file or directory: '../input/train.csv'